In [72]:
import pandas as pd

In [73]:
df = pd.read_csv('pivoted_preprocessed_dataset.csv')
df.shape

(987, 18)

In [74]:
display(df)

,Unnamed: 0,Reference area,TIME_PERIOD,Adult literacy skills [Points],Average annual gross earnings [ PPP converted],Difficulty making ends meet [Percentage of population aged 16 years or over],Employment rate [Percentage of population aged 25-64 years],Feeling lonely [Percentage of population aged 16 years or over],Feeling safe at night [Percentage of population aged 15 years or over],Having a say in government [Percentage of population aged 16-65 years],Households and NPISHs net adjusted disposable income per capita [ PPP converted],Housing affordability [Percentage of household gross adjusted disposable income],Life expectancy at birth [Years],Life satisfaction [0-10 scale],Long-term unemployment rate [Percentage of labour force],Self-reported depression [Percentage of population aged 15 years or over],Social support [Percentage of population aged 15 years or over],Student reading skills [Points]
0,0,Argentina,2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.9,NaN,NaN,NaN,NaN,NaN
1,1,Argentina,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.2,NaN,NaN,NaN,NaN,NaN
2,2,Argentina,2006,NaN,NaN,NaN,NaN,NaN,41.500000,NaN,NaN,NaN,75.3,NaN,NaN,NaN,89.013534,NaN
3,3,Argentina,2007,NaN,NaN,NaN,NaN,NaN,41.500000,NaN,NaN,NaN,74.8,NaN,NaN,NaN,89.013534,NaN
4,4,Argentina,2008,NaN,NaN,NaN,NaN,NaN,39.333333,NaN,NaN,NaN,75.4,NaN,NaN,NaN,90.156300,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,982,United States,2020,NaN,82783.413,NaN,71.795,NaN,74.666667,NaN,61133.0,82.758441,77.0,NaN,0.455155,NaN,91.784653,NaN
983,983,United States,2021,NaN,84211.210,NaN,73.680,NaN,74.666667,NaN,63022.0,82.821431,76.4,NaN,1.252592,NaN,91.784653,NaN
984,984,United States,2022,NaN,81837.833,NaN,75.803,NaN,74.666667,NaN,59303.0,81.509138,77.5,NaN,0.556768,NaN,91.784653,503.94
985,985,United States,2023,259.294451,82078.428,NaN,76.641,NaN,71.500000,NaN,61353.0,81.753107,78.4,NaN,0.429940,NaN,89.914042,NaN


In [75]:
df['TIME_PERIOD'].unique()

array([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024],
      dtype=int64)

In [76]:
df = df.loc[:, df.isna().sum() <= 700]

In [77]:
import numpy as np

def fill_series_with_quantiles(s_year, s_val):
    """
    s_year: pandas Series of years (sorted ascending)
    s_val: pandas Series of values aligned with s_year (may contain NaN)
    Returns a Series of filled values (same index as s_val).
    """
    values = s_val.copy()

    # indices of non-missing values
    known_idx = np.where(~values.isna())[0]
    if len(known_idx) < 2:
        # nothing to interpolate between; return as is
        return values

    years = s_year.values

    for i in range(len(known_idx) - 1):
        left_idx = known_idx[i]
        right_idx = known_idx[i+1]

        y0, v0 = years[left_idx], values.iloc[left_idx]
        y1, v1 = years[right_idx], values.iloc[right_idx]

        gap = (y1 - y0 - 1)
        if gap <= 0:
            continue

        # positions of missing years in this gap (by index)
        missing_positions = range(left_idx + 1, right_idx)

        if gap == 1:
            # simple mean
            fill_val = (v0 + v1) / 2.0
            values.iloc[missing_positions[0]] = fill_val
        else:
            # use quantiles of the change
            delta = v1 - v0
            # quantiles 1/(gap+1), 2/(gap+1), ..., gap/(gap+1)
            qs = np.arange(1, gap + 1) / (gap + 1)
            for pos, q in zip(missing_positions, qs):
                values.iloc[pos] = v0 + q * delta

    return values

In [78]:
df['TIME_PERIOD'].value_counts().sort_index()

TIME_PERIOD
2004    47
2005    47
2006    47
2007    47
2008    47
2009    47
2010    47
2011    47
2012    47
2013    47
2014    47
2015    47
2016    47
2017    47
2018    47
2019    47
2020    47
2021    47
2022    47
2023    47
2024    47
Name: count, dtype: int64

In [79]:


# ensure sorted by country then year
df = df.sort_values(["Reference area", "TIME_PERIOD"])

numeric_cols = [
    c for c in df.columns
    if c not in ["Reference area", "TIME_PERIOD"] and pd.api.types.is_numeric_dtype(df[c])
]

def fill_group(group):
    group = group.sort_values("TIME_PERIOD")
    years = group["TIME_PERIOD"]
    for col in numeric_cols:
        group[col] = fill_series_with_quantiles(years, group[col])
    return group

df_filled = df.groupby("Reference area", group_keys=False).apply(fill_group)

C:\Users\sasha\AppData\Local\Temp\ipykernel_35032\2120221087.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filled = df.groupby("Reference area", group_keys=False).apply(fill_group)


In [86]:
df_pro = df_filled[df_filled.isna().sum(axis=1) <= 3]
df_pro['TIME_PERIOD'].value_counts().sort_index()




TIME_PERIOD
2004    31
2005    33
2006    38
2007    38
2008    39
2009    39
2010    39
2011    39
2012    40
2013    40
2014    40
2015    40
2016    40
2017    40
2018    41
2019    41
2020    41
2021    41
2022    41
2023    41
2024    25
Name: count, dtype: int64